# Video Transformer

In [15]:
from timm.models.vision_transformer import VisionTransformer, PatchEmbed, HybridEmbed
from torch import nn
from timm.models.layers.helpers import to_2tuple
import torch
from math import prod
from fastcore.all import *

In [25]:
class PatchEmbed3D(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=(16, 112, 112), patch_size=(4, 16, 16), in_chans=1, embed_dim=768):
        super().__init__()
        store_attr()
        self.num_patches = prod(patch_size)

        self.proj = nn.Conv3d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, D, H, W = x.shape
        # FIXME look at relaxing size constraints
        assert H == self.img_size[1] and W == self.img_size[2], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[1]}*{self.img_size[2]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x


In [38]:
m = VisionTransformer(img_size = 224, patch_size = 8, in_chans = 1, num_classes = 2)

In [35]:
PatchEmbed(img_size = 224, patch_size = 8)(torch.randn(1,3,224,224)).shape

torch.Size([1, 784, 768])

In [40]:
PatchEmbed3D(img_size = (16, 224, 224))(torch.randn(1,1,16,224,224)).shape

torch.Size([1, 784, 768])

In [31]:
m = VisionTransformer(img_size = 112, patch_size = 16, in_chans = 1, num_classes = 2)
m.patch_embed = PatchEmbed3D()